In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pickle
import keras
from keras import layers
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback
from keras.applications.vgg19 import VGG19

folder = "/content/drive/MyDrive/Colab Notebooks/Zama"
num_classes = 80
batch_size = 16

# load embedding model
siamese_embedding_model = keras.models.load_model(f'{folder}/embedding_network_512.h5')     #  USAGE: res = siamese_embedding_model.predict( tf_ds )

In [ ]:
from datasets import load_dataset, DatasetDict

def custom_train_test_split( spec ):

    # Split into:
    # train      90%
    # validation  5%
    # test        5%
    # following https://stackoverflow.com/questions/76001128/splitting-dataset-into-train-test-and-validation-using-huggingface-datasets-fun
    spec = spec.train_test_split(test_size=0.1)

    data_train = spec['train']
    data_val_test = spec['test']

    data_val_test = data_val_test.train_test_split(test_size=0.5)

    return (data_train, data_val_test['train'], data_val_test['test'] )

#### LIbraries for model creation

In [ ]:
# image dimensions
rows = 224
cols = 224

def create_cls_model_fn( num_classes):
    # create a vgg19 model and discard the top layer
    model_vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(rows, cols, 3))
    model_vgg19.trainable = False

    image = keras.Input(shape=(224, 298, 3))
    embedding = keras.Input(shape=(512))

    x = layers.RandomCrop(rows, cols)(image)
    x = layers.Rescaling(scale=1.0 / 127.5, offset=-1)(x)

    x = model_vgg19(x)

    x = layers.Flatten()(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2048, activation='relu')(x)

    outputs = keras.layers.Concatenate()([x, embedding])
    outputs = layers.Dense(num_classes, activation='softmax')(outputs)

    model = keras.Model(inputs=[image, embedding], outputs=outputs, name='cluster_classification')

    # Compile model
    model.compile(
        optimizer= keras.optimizers.SGD(learning_rate=0.005),
        loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False),

        metrics = [keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

### Cluster classification

In [ ]:
# A spectrogram images dataset for classification of tracks per cluster
cluster_cls_ds_handle = '8000_large_4_siamclusters'    #'siamese_clusters'                                #  '8000_large_4_siamclusters'
spec = load_dataset(f"arieg/{cluster_cls_ds_handle}")
spec.set_format('tf')

# Split into train / eval / test
ds_train, ds_eval, ds_test = custom_train_test_split(spec['train'])

# Train
image_train = ds_train['image'].numpy()
emb_train = siamese_embedding_model.predict( ds_train.with_format('tf').to_tf_dataset(columns="image", batch_size=batch_size))
label_train = np.array( ds_train['siam_cluster'] ).astype("float32")
print(image_train.shape, emb_train.shape, label_train.shape)

# Eval
image_eval = ds_eval['image'].numpy()
emb_eval = siamese_embedding_model.predict( ds_eval.with_format('tf').to_tf_dataset(columns="image", batch_size=batch_size))
label_eval = np.array( ds_eval['siam_cluster'] ).astype("float32")
print(image_eval.shape, emb_eval.shape, label_eval.shape)

# Test
image_test = ds_test['image'].numpy()
emb_test = siamese_embedding_model.predict( ds_test.with_format('tf').to_tf_dataset(columns="image", batch_size=batch_size))
label_test = np.array( ds_test['siam_cluster'] ).astype("float32")
print(image_test.shape, emb_test.shape, label_test.shape)

# Make a classification model for this cluster
model = create_cls_model_fn( num_classes )

# Training
epochs = 20
history = model.fit([image_train, emb_train], label_train, epochs=epochs, validation_data=([image_eval, emb_eval], label_eval), callbacks=[], verbose=1)

# save training history and model
with open(f'{folder}/8000_large_4_siamclusters_enhanced.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

model.save(f'{folder}/8000_large_4_siamclusters_enhanced.keras')